In [ ]:
%matplotlib inline
import stumpy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle
import datetime
import os
import random

from sklearn.model_selection import train_test_split

In [ ]:
from utils import Utils
utils = Utils()

### Band

In [ ]:
band="theta"
utils.read_data(band=band)

In [ ]:
all_sequences = utils.get_all_patient_signals(length=n)[0:l]
labels = np.array(utils.get_labels(labeling=1))

final_test_sequences = utils.get_all_patient_signals(dataset="test")
final_test_labels = np.array(utils.get_labels(labeling=1, dataset="test"))

gender_info = np.array(utils.get_genders(dataset="train"))
test_gender = np.array(utils.get_genders(dataset="test"))

In [ ]:
str_now = str(datetime.datetime.now()).replace(" ", "").replace("-","").replace(":", "").replace(".", "")[:-8]
str_now = "ostinato_" + str_now + "_" + band
os.mkdir(str_now)
print("Created directory:", str_now)

In [ ]:
def get_balanced_split(all_sequences, labels, gender_info):

    train_sequences = []
    validation_sequences = []
    train_labels = []
    validation_labels = []
    train_gender = []
    validation_gender = []

    for i in [0, 1]:
        print("Gender:", i)
        gender_ind = list(np.argwhere(gender_info == i).T[0])

        all_sequences_ = [sequences for j, sequences in enumerate(all_sequences) if j in gender_ind]
        print(len(all_sequences_))
        labels_ = [label for j, label in enumerate(labels) if j in gender_ind]
        labels_ = np.array(labels_)

        split_ratio = 0.15
        if i == 1:
            split_ratio = 0.3

        train_sequences_, validation_sequences_, train_labels_, validation_labels_, train_ind, val_ind = train_test_split(all_sequences_, labels_, gender_ind, stratify=labels_, test_size=split_ratio, random_state=2)
        print("--Train:", len(train_sequences_))
        print("patient indexes:", train_ind)
        print("----", sum(train_labels_ == 0))
        print("----", sum(train_labels_ == 1))
        print("--Val:", len(validation_sequences_))
        print("patient indexes:", val_ind)
        print("----", sum(validation_labels_ == 0))
        print("----", sum(validation_labels_ == 1))

        train_sequences.append(train_sequences_)
        validation_sequences.append(validation_sequences_)
        train_labels.append(list(train_labels_))
        validation_labels.append(list(validation_labels_))
        train_gender.append([i]*len(train_sequences_))
        validation_gender.append([i]*len(validation_sequences_))

    # flatten
    train_sequences = sum(train_sequences, [])
    validation_sequences = sum(validation_sequences, [])
    train_labels = np.array(sum(train_labels, []))
    validation_labels = np.array(sum(validation_labels, []))
    train_gender = sum(train_gender, [])
    validation_gender = sum(validation_gender, [])
    
    return train_sequences, validation_sequences, train_labels, validation_labels, train_gender, validation_gender

### Plotting methods

In [ ]:
def plot_consensus_motifs(sequences, Ts_idx, subseq_idx):
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 10
    fig_size[1] = 4
    plt.rcParams["figure.figsize"] = fig_size
    found_motifs = []

    seed_motif = sequences[Ts_idx][subseq_idx : subseq_idx + m]
    x = np.linspace(0,1,m)
    nn = np.zeros(len(sequences), dtype=np.int64)
    nn[Ts_idx] = subseq_idx
    for i, e in enumerate(sequences):
        if i != Ts_idx:
            nn[i] = np.argmin(stumpy.core.mass(seed_motif, e))
            lw = 1
            label = None
        else:
            lw = 6
            label = "ind" + str(i+1) + ' (seed)'
        plt.plot(x, e[nn[i]:nn[i]+m], lw=lw, label=label)
        found_motifs.append(e[nn[i]:nn[i]+m])
    plt.title('The Consensus Motif')
    plt.xlabel('Time')
    plt.legend()
    plt.show()
    
    return found_motifs

In [ ]:
def plot_distances_to_seed(seed_motif, sequences_0, sequences_1):
    distances_0 = []
    distances_1 = []
    
    for sequence in sequences_0:
        matches = stumpy.match(seed_motif, sequence, max_distance=None, max_matches=2, normalize=True, p=2.0)
        dist = np.min(matches[:, 0])
        distances_0.append(dist)
        
    for sequence in sequences_1:
        matches = stumpy.match(seed_motif, sequence, max_distance=None, max_matches=2, normalize=True, p=2.0)
        dist = np.min(matches[:, 0])
        distances_1.append(dist)
        
    distances_0 = sorted(distances_0)
    distances_1 = sorted(distances_1)
    plt.plot(distances_0, label="class_0")
    plt.plot(distances_1, label="class_1")
    plt.legend()
    plt.show()

### Data generating/saving methods

In [ ]:
def generate_feature_matrix(input_sequences, input_motifs, input_electrodes):
    X = []
    # iterate over given patients
    for j, sequences in enumerate(input_sequences):
        print("Patient", j+1)
        patient_motif_distances = []
        # iterate over all discovered motifs
        for i, motif in enumerate(input_motifs):
            electrode = input_electrodes[i]

            signal = sequences[electrode]
            matches = stumpy.match(motif, signal, max_distance=None, max_matches=3, normalize=True, p=2.0)
            if len(matches) == 0:
                patient_motif_distances.append(np.nan)
            else:
                patient_motif_distances.append(np.mean(matches[:, 0]))
        X.append(patient_motif_distances)
    return X

In [ ]:
def save_feature_matrices(m, final_motifs, final_electrodes, motif_names, path='./feature_matrices/'):
    X_train = generate_feature_matrix(train_sequences, final_motifs, final_electrodes)
    print(len(X_train))
    print(len(X_train[0]))

    X_validation_final = generate_feature_matrix(validation_sequences, final_motifs, final_electrodes)
    print(len(X_validation_final))
    print(len(X_validation_final[0]))
    
    X_test_final = generate_feature_matrix(final_test_sequences, final_motifs, final_electrodes)
    print(len(X_test_final))
    print(len(X_test_final[0]))

    df_train = pd.DataFrame(data = np.array(X_train), columns = motif_names)
    df_train["label"] = train_labels
    print(df_train.head())
    df_train.to_csv(path+"motifs_{}_m{}_train.csv".format(band, m), index=False)
    print("Writing... train feature matrix")

    df_validation = pd.DataFrame(data = np.array(X_validation_final), columns = motif_names)
    df_validation["label"] = validation_labels
    print(df_validation.head())
    df_validation.to_csv(path+"motifs_{}_m{}_val.csv".format(band, m), index=False)
    print("Writing... validation feature matrix")
    
    df_test = pd.DataFrame(data = np.array(X_test_final), columns = motif_names)
    df_test["label"] = final_test_labels
    print(df_test.head())
    df_test.to_csv(save_path + "motifs_ostinato{}test_m={}.csv".format(name, m), index=False)
    print("Writing... testing feature matrix")

In [ ]:
def save_motif_data(m, all_motifs, seed_motif_patient, seed_motif_idx, all_electrodes, all_genders, all_classes, all_diff_scores, save_path="./", name="_"):
    df_motifs = pd.DataFrame()
    df_motifs["motif"] = all_motifs
    df_motifs["train_ind"] = seed_motif_patient
    df_motifs["indexes"] = seed_motif_idx
    df_motifs["electrode"] = all_electrodes
    df_motifs["gender"] = all_genders
    df_motifs["class"] = all_classes
    df_motifs["diff_score"] = all_diff_scores
    print(df_motifs.head())
    df_motifs.to_csv(save_path + "motifs_ostinato_exp{}m={}.csv".format(name, m), index=False)

### Filter out motifs that are similarly present (heuristics) among the two classes

In [ ]:
def compute_distances_to_classes(motif, input_sequences, input_labels, electrode, index):
    
    class_0_ind = list(np.argwhere(input_labels == 0).T[0])
    class_1_ind = list(np.argwhere(input_labels == 1).T[0])
    
    distances = []
    for j, sequences in enumerate(input_sequences):
        #print("Patient", j+1)
        signal = sequences[electrode]

        matches = stumpy.match(motif, signal, max_distance=None, max_matches=10, normalize=True, p=2.0)
        if len(matches) == 0:
            distances.append(np.nan)
        else:
            dist = np.mean(matches[:, 0])
            distances.append(dist)
        
    perc = int(len(input_sequences)/3)
    
    class_0_distances = [distances[i] for i in class_0_ind if round(distances[i]) > 0 and i!=index]
    class_0_distances_ = sorted(class_0_distances)[0:perc]
    class_1_distances = [distances[i] for i in class_1_ind if round(distances[i]) > 0 and i!=index]
    class_1_distances_ = sorted(class_1_distances)[0:perc]
    
    return class_0_distances, class_1_distances, class_0_distances_, class_1_distances_ 

###  Run Ostinato

#### 1. Group electrodes by the class

In [ ]:
# Separate the sets by the labels
sequence_dict = dict({
    "class_0": dict({}),
    "class_1": dict({})
})

for i in range(19):
    sequence_dict["class_0"][i] = []
    sequence_dict["class_1"][i] = []
    for j in range(len(train_sequences)):
        if train_labels[j] == 0:
            sequence_dict["class_0"][i].append(train_sequences[j][i])
        else:
            sequence_dict["class_1"][i].append(train_sequences[j][i])

In [ ]:
for m in [50, 100, 250, 500, 1000, 2000]:
    print(m)
    all_motifs = []
    all_electrodes = []

    seed_motif_patient = []
    seed_motif_idx = []

    motif_names = []

    for class_label in ["class_0", "class_1"]:
        print("Class:", class_label)

        for el in range(19):
            print("Electrode:", el)
            sequences = sequence_dict[class_label][el]

            radius, Ts_idx, subseq_idx = stumpy.ostinato(sequences, m)
            print(f'Found Best Radius {np.round(radius, 2)} in time series {Ts_idx} starting at subsequence index location {subseq_idx}.')
            #found_motifs = plot_consensus_motifs(sequences, Ts_idx, subseq_idx)

            seed_motif = sequences[Ts_idx][subseq_idx : subseq_idx + m]

            all_motifs.append(seed_motif)
            all_electrodes.append(el)

            seed_motif_patient.append(Ts_idx)
            seed_motif_idx.append(subseq_idx)

            motif_names.append("{}_el_{}".format(class_label, el))

            #plot_distances_to_seed(seed_motif, sequence_dict["class_0"][el], sequence_dict["class_1"][el])

    # save result
    save_motif_data(m, seed_motif_patient, seed_motif_idx, all_electrodes, save_path=str_now + "/")
    save_feature_matrices(m, all_motifs, all_electrodes, motif_names, path=str_now + "/")

#### 2. Group electrodes by the labels and gender

In [ ]:
# Separate the sets by the labels
sequence_dict = dict({
    "class_0": dict({
        "gender_0": dict({}),
        "gender_1": dict({}),
    }),
    "class_1": dict({
        "gender_0": dict({}),
        "gender_1": dict({}),
    })
})

# initialize
for i in range(19):
    for class_ in ["class_0", "class_1"]:
        for gender_ in ["gender_0", "gender_1"]:
            sequence_dict[class_][gender_][i] = []


for i in range(19):
    for j in range(len(train_sequences)):
        sequence_dict["class_{}".format(train_labels[j])]["gender_{}".format(train_gender[j])][i].append(train_sequences[j][i])

In [ ]:
for class_ in ["class_0", "class_1"]:
    for gender_ in ["gender_0", "gender_1"]:
        print(class_, gender_)
        print(len(sequence_dict[class_][gender_][0]))

In [ ]:
for m in [50, 100, 250, 500, 1000, 2000]:
    print(m)
    all_motifs = []
    all_electrodes = []
    all_genders = []
    all_classes = []
    all_diff_scores = []
    
    seed_motif_patient = []
    seed_motif_idx = []

    motif_names = []

    for class_label in ["class_0", "class_1"]:
        print("Class:", class_label)
        for gender in ["gender_0", "gender_1"]:
            print("Gender:", gender)

            for el in range(19):
                for repetition in range(3):
                    print("Electrode:", el)
                    sequences = sequence_dict[class_label][gender][el]
                    
                    # randomly choose the sequences
                    chosen_sequences = random.sample(sequences, int(len(sequences)/3))
                    print(len(chosen_sequences))

                    radius, Ts_idx, subseq_idx = stumpy.ostinato(chosen_sequences, m)
                    print(f'Found Best Radius {np.round(radius, 2)} in time series {Ts_idx} starting at subsequence index location {subseq_idx}.')
                    found_motifs = plot_consensus_motifs(chosen_sequences, Ts_idx, subseq_idx)

                    seed_motif = chosen_sequences[Ts_idx][subseq_idx : subseq_idx + m]
                    
                    class_0_distances, class_1_distances, class_0_distances_, class_1_distances_ = compute_distances_to_classes(seed_motif, train_sequences, train_labels, el)
                    diff = np.abs(np.mean(class_0_distances) - np.mean(class_1_distances))
                    diff_ = np.abs(np.mean(class_0_distances_) - np.mean(class_1_distances_))
                    print(diff_)
                    
                    all_diff_scores.append(diff_)
                    all_motifs.append(seed_motif)
                    all_electrodes.append(el)
                    all_genders.append(gender)
                    all_classes.append(class_label)

                    seed_motif_patient.append(Ts_idx)
                    seed_motif_idx.append(subseq_idx)

                    motif_names.append("{}_{}_el_{}".format(class_label, gender, el))

                    #plot_distances_to_seed(seed_motif, sequence_dict["class_0"][el], sequence_dict["class_1"][el])

    # save result
    save_motif_data(m, all_motifs, seed_motif_patient, seed_motif_idx, all_electrodes, all_genders, all_classes, all_diff_scores, save_path=str_now + "/", name="_gender_")
    save_feature_matrices(m, all_motifs, all_electrodes, motif_names, save_path=str_now + "/")